## Ranking D3.js packages (libraries, plugins, and utilities)

In [ ]:
import requests
import logging
import pandas as pd
import numpy as np
import urllib2
import dill
import time
import json
from datetime import date, timedelta
from bs4 import BeautifulSoup
import unicodedata
import csv
import sys
sys.path.append('../code')

## Get D3 packages list
https://github.com/wbkd/awesome-d3

In [2]:
r1 = requests.get('https://github.com/wbkd/awesome-d3')
soup1 = BeautifulSoup(r1.text, 'lxml')
D3_packages = []

In [14]:
D3_packages = ['billboard', 'britecharts', 'bullet chart', 'cirrus.js', 'cubism', 'colajs', 'c3', 'dagre-d3', 'dc', 'dimple', 'd3.chart', 'd3.chart.sankey', 'd3-bar', 'd3-bumps-chart', 'd3-circle', 'd3-dot', 'd3-heatmap', 'd3-line', 'd3-flame-graph', 'd3-horizon-chart', 'd3-message-sequence', 'd3-timeline', 'd3fc', 'd3-funnel', 'd3panels', 'd3pie', 'd3xter', 'd3 timeseries', 'd4', 'dtree', 'epoch', 'eventDrops', 'fancycharts', 'firespray', 'forest-d3', 'gantt-chart', 'insights', 'metrics-graphics', 'micropolar', 'mpld3', 'nvd3', 'peek', 'plotly', 'plottable', 'pykcharts', 'radar chart', 'rickshaw', 'sankey', 'taucharts', 'techan', 'uvcharts', 'vega', 'vizabi', 'xkcdplot', 'angular-nvd3', 'd3act', 'd3 simplecharts', 'ember charts', 'n3-charts', 'react-d3', 'react-d3-components', 'react-d3-library', 'react-stockcharts', 'react-vis', 'recharts', 'victory', 'vx', 'd3.chart', 'koto', 'carto map', 'd3-composite-projections', 'datamaps', 'd3 cartogram', 'd3 exploder', 'd3 geo projection', 'd3 geomap', 'd3.geo2rect', 'mapmap', 'mapsense.js', 'maptable', 'simple map', 'spam', 'leaflet-d3', 'react d3 map', 'wikimaps-d3js atlas', 'chroniton', 'd3kit-timeline', 'd3scription', 'd3-annotate', 'd3-area-label', 'd3-component', 'd3-extended', 'd3-helpers', 'd3-history', 'd3-iconarray', 'd3kit', 'd3-labeler', 'd3-lasso', 'd3-legend', 'd3-loom', 'd3-nelson-rules', 'd3-peaks', 'd3-ring-note', 'd3-scale-cluster', 'd3-sparql', 'd3-starterkit', 'd3-tip', 'd3-tipy', 'd3-trail', 'd3-xray', 'd3slider', 'd3-svg-circularbrush', 'd3-svg-ribbon', 'iopctrl', 'jetpack', 'kodama', 'swoopyarrows', 'd3-swoopy-drag', 'textures', 'chess-dataviz', 'codeflower', 'comic.js', 'dragit', 'd3moji', 'd3-cloud', 'd3-context-menu', 'd3-force-gravity', 'd3-resume', 'd3.sketchy', 'd3plus', 'd3plus-text', 'graph-scroll', 'jsdoc2diagram', 'mermaid', 'netvis']

In [15]:
print D3_packages

['billboard', 'britecharts', 'bullet chart', 'cirrus.js', 'cubism', 'colajs', 'c3', 'dagre-d3', 'dc', 'dimple', 'd3.chart', 'd3.chart.sankey', 'd3-bar', 'd3-bumps-chart', 'd3-circle', 'd3-dot', 'd3-heatmap', 'd3-line', 'd3-flame-graph', 'd3-horizon-chart', 'd3-message-sequence', 'd3-timeline', 'd3fc', 'd3-funnel', 'd3panels', 'd3pie', 'd3xter', 'd3 timeseries', 'd4', 'dtree', 'epoch', 'eventDrops', 'fancycharts', 'firespray', 'forest-d3', 'gantt-chart', 'insights', 'metrics-graphics', 'micropolar', 'mpld3', 'nvd3', 'peek', 'plotly', 'plottable', 'pykcharts', 'radar chart', 'rickshaw', 'sankey', 'taucharts', 'techan', 'uvcharts', 'vega', 'vizabi', 'xkcdplot', 'angular-nvd3', 'd3act', 'd3 simplecharts', 'ember charts', 'n3-charts', 'react-d3', 'react-d3-components', 'react-d3-library', 'react-stockcharts', 'react-vis', 'recharts', 'victory', 'vx', 'd3.chart', 'koto', 'carto map', 'd3-composite-projections', 'datamaps', 'd3 cartogram', 'd3 exploder', 'd3 geo projection', 'd3 geomap', 'd3.

In [21]:
len(D3_packages)

135

In [94]:
pkgsDF = pd.read_csv('../data/D3_pkgs.csv')

## Get Github forks and stars

In [52]:
pkgsDF.head()

,package,github_repo,tags,search_terms
0,billboard,mick/billboard,[billboard],[billboard-d3]
1,britecharts,eventbrite/britecharts,[britecharts],[britecharts]
2,bullet chart,stefanwalther/qvD3BulletCharts,[bullet-chart],[bullet-chart]
3,cirrus.js,NaN,[cirrus.js],[cirrus.js]
4,cubism,square/cubism,[cubism.js],[cubism]


In [17]:
with open("../code/secrets/github-token.nogit", "rb") as f:
    token = f.read()

In [18]:
headers = {'Authorization': 'token %s' % token}

In [76]:
def get_data_from_github(repo):
    """Use github repo from DF to return stars and forks"""
    if str(repo)== 'nan':
        return {'github_repo': repo, 'stars': 'NA', 'forks': 'NA'}
      
    r = requests.get('https://api.github.com/repos/'+str(repo),
                     headers=headers)
    r.raise_for_status()
    try:
        res = r.json()
        return {'github_repo': repo, 'stars': res['stargazers_count'], 'forks': res['forks_count']}
    except:
        return {'github_repo': repo, 'stars': 'NA', 'forks': 'NA'}

In [51]:
repoList = pkgsDF['github_repo'].tolist()

In [78]:
# use generator to avoid repeat API calls
data1 = [res for res in (get_data_from_github(q) for q in repoList[:29])
        if res is not None]
time.sleep(61)      # API limit with token: 30 api calls/min
data2 = [res for res in (get_data_from_github(q) for q in repoList[29:57])
        if res is not None]
time.sleep(61)
data3 = [res for res in (get_data_from_github(q) for q in repoList[57:85])
        if res is not None]
time.sleep(61)
data4 = [res for res in (get_data_from_github(q) for q in repoList[85:104])
        if res is not None]
time.sleep(61)
data5 = [res for res in (get_data_from_github(q) for q in repoList[104:])
        if res is not None]
print "DONE"

DONE


In [79]:
githubDF = pd.DataFrame(data1 + data2 + data3 + data4 + data5)[['github_repo', 'stars', 'forks']]

In [104]:
pkgsDF['forks'] = githubDF['forks']
pkgsDF['stars'] = githubDF['stars']

In [41]:
github.to_csv("../data/D3_pkgs.csv", columns = ['package','full_name'], index= False)

In [ ]:
with open("../data/D3_pkgs.csv", "w") as f:
        writer = csv.writer(f)
        for k,v in github_names_dict.iteritems():
            writer.writerow([k] + [v])

## Get StackOverflow tags and questions

In [26]:
baseurl = 'https://api.stackexchange.com/2.2/search/advanced'

def get_so_tag_counts(tag):
#Given tag, return tag counts
    params = {
    "site": "stackoverflow",
    "key": "y38PeNERQJQIC8EPliKAVQ((",
    "tagged": tag,  
    "filter": 'total'}
    try:
        r = requests.get(baseurl, params=params)
        return r.json()['total']
    except:
        return 0

def get_so_question_counts(package):
#Given package, return count of questions containing package name
    params = {
    "site": "stackoverflow",
    "key": "y38PeNERQJQIC8EPliKAVQ((",
    "q": package,  
    "filter": "total"}
    try:
        r = requests.get(baseurl, params=params)
        return r.json()['total']
    except:
        return 0    

In [89]:
pkgTags = pkgsDF['tags'].tolist()

In [90]:
tag_data = []
for tags in pkgTags:
    total_count = 0
    for alias in tags.split(';'):
        total_count += get_so_tag_counts(alias)
    tag_data.append({'tags':tags, 'so_tag_count': total_count}) 

In [102]:
so_tagsDF = pd.DataFrame(tag_data)[['tags', 'so_tag_count']]
so_tagsDF.columns = ['tags', 'so_tag_counts']
pkgsDF['so_tag_counts'] = so_tagsDF['so_tag_counts']

In [95]:
pkgTerms = pkgsDF['search_terms'].tolist()

In [96]:
question_data = []
for terms in pkgTerms:
    total_count = 0
    for term in terms.split(';'):
        total_count += get_so_question_counts(term)
    question_data.append({'search_terms':terms, 'so_question_count': total_count})

In [97]:
so_questDF = pd.DataFrame(question_data)[['search_terms', 'so_question_count']]
so_questDF.columns = ['search_terms', 'so_question_count']
pkgsDF['so_questions_counts'] = so_questDF['so_question_count']

## Get download counts from npmjs

In [98]:
def extract_count(html):
    total_dl = 0
    day_queries = json.loads(html).get(u'downloads')
    for day in day_queries:
        total_dl += day.get(u'downloads')
    return total_dl

def get_npmjs_counts(package, start_date, end_date):
    #retruns total downloads between dates
    #enter package and dates as strings, date format YYYY-MM-DD
    try:
        response = urllib2.urlopen('https://api.npmjs.org/downloads/range/'+start_date+':'+end_date+'/'+package)
        return extract_count(response.read())
    except urllib2.HTTPError, e:
        return 'NA'

def get_npmjs_cmgr(package, first_month, last_month):
    #returns compound monthly growth rate
    #enter package as string, date as list of two integers [YYYY, M]
    try:
        d1 = date(first_month[0], first_month[1], 1)
        d2 = date(last_month[0], last_month[1], 1)
        month_diff = (d2.year - d1.year) * 12 + d2.month - d1.month + 1
        d1_end = d1 + timedelta(days=30)
        month1_counts = extract_count(urllib2.urlopen('https://api.npmjs.org/downloads/range/'
                                                  +d1.strftime("%Y-%m-%d")+':'+d1_end.strftime("%Y-%m-%d")+'/'+package).read())
        d2_end = d2 + timedelta(days=30)
        month2_counts = extract_count(urllib2.urlopen('https://api.npmjs.org/downloads/range/'
                                                  +d2.strftime("%Y-%m-%d")+':'+d2_end.strftime("%Y-%m-%d")+'/'+package).read())
        if month1_counts == 0:
            return 'NA'
        return (((month2_counts/float(month1_counts))**(1.0/month_diff))-1.0)
    except urllib2.HTTPError, e:
        return 'NA'
    

def get_npmjs_stats(npm_package_list):
    #returns downloads in last 3 years, and 6 month compound monthly growth rate
    downloads =[]
    growth_rates = []
    for package in npm_package_list:
        downloads.append(get_npmjs_counts(package, '2014-08-01','2017-07-31'))
        growth_rates.append(get_npmjs_cmgr(package, [2017, 2], [2017, 7]))
    return downloads, growth_rates

In [99]:
pkgs = pkgsDF['package']
[downloads, growth_rates] = get_npmjs_stats(pkgs)
downloadsDF = pd.DataFrame(np.column_stack([pkgs, downloads, growth_rates]), 
                               columns=['package', 'downloads', 'growth_rates'])
pkgsDF['downloads'] = downloadsDF['downloads']
pkgsDF['dl_growth_rates'] = downloadsDF['growth_rates']

## All data combined

In [106]:
pkgsDF.to_csv("../output/D3_pkgs_data.csv")

In [105]:
pkgsDF.head()

,package,github_repo,tags,search_terms,so_questions_counts,downloads,dl_growth_rates,so_tag_counts,forks,stars
0,billboard,mick/billboard,[billboard],[billboard-d3],0,445,0.064440073412,0,6,12
1,britecharts,eventbrite/britecharts,[britecharts],[britecharts],0,6445,1.10558106204,0,91,2571
2,bullet chart,stefanwalther/qvD3BulletCharts,[bullet-chart],[bullet-chart],11,584,-0.0427851196966,11,6,7
3,cirrus.js,NaN,[cirrus.js],[cirrus.js],0,NA,NA,0,NA,NA
4,cubism,square/cubism,[cubism.js],[cubism],0,6309,-0.057776985543,92,541,4554
